In [1]:
import os
import sys
#import utils
import pandas as pd

In [20]:
file_in = "GSE49757_Septic_vs_Healthy.txt"
gmt_file = "Reactome_2016_15and100Genes.gmt"
deg_list = pd.read_csv(file_in, sep = "\t")
deg_list = deg_list.dropna()
list(deg_list["Gene.symbol"])

['P2RY2',
 'METTL6',
 'INTS9',
 'TM9SF4',
 'TPST1',
 'SPPL2A',
 'MCRIP2',
 'PRPSAP1',
 'CCND3',
 'TNIP1',
 'TNFRSF9',
 'SRC',
 'P2RY2',
 'COL13A1',
 'IRS2',
 'STK17B',
 'CYB5R1',
 'KIF3B',
 'SAR1B',
 'IL18R1',
 'RUNX3',
 'CADM4',
 'TCFL5',
 'FMNL3',
 'H2AFY',
 'B9D2',
 'POLR2H',
 'CEBPD',
 'EGLN3',
 'UBALD2',
 'KLF6',
 'CXCL16',
 'ADIPOR1',
 'TSC22D1',
 'CLEC4A',
 'CERK',
 'MPP1',
 'BMP6',
 'TNPO1',
 'SAR1B',
 'BCL11A',
 'RGS1',
 'SIPA1L2',
 'UBE2E1',
 'ADORA3',
 'CD7',
 'TICAM2',
 'SEC24A',
 'TSC22D1',
 'XPO6',
 'CFP',
 'MCTP1',
 'PGAM4',
 'COLEC12',
 'TSC22D3',
 'SLC25A19',
 'RXRA',
 'TBC1D22A',
 'CLIC4',
 'H2AFY',
 'ADIPOR1',
 'H2AFY',
 'ABCG1',
 'PDE6D',
 'ABCG1',
 'MGST2',
 'C17orf49',
 'ZNF770',
 'LOC407835',
 'BRPF3',
 'CLDN14',
 'NFKB1',
 'VLDLR',
 'IFT88',
 'WDR61',
 'OSM',
 'AMPD3',
 'PKM',
 'ZCCHC6',
 'APBA3',
 'SLC36A1',
 'FBXW7',
 'DNAJC10',
 'SLC35E1',
 'HDGF',
 'RNF13',
 'PGAM1',
 'CRTC2',
 'NLRP6',
 'ID2',
 'SEC61G',
 'NEDD4',
 'DLGAP1-AS1',
 'PGAM1',
 'EML2',
 'MMP9',


In [ ]:
def pebba(file_in, 
          gmt_file, 
          gene_col="Gene.symbol",
          logFC_col="logFC",
          pvalue_col="P.Value",
          min_genes=100,
          max_genes=1500,
          p_cut=0.2,
          verbose=True,
          analysis_name= None, 
          results_dir="Results",
          force=False)

    validates_inputs(min_genes,max_genes,p_cut)

    create_results_directory(results_dir)

    

    
    ## Get information from all unique terms
    term2gene , path_desc , merge_p  = utils.read_gmt_hier(gmt_file)    
# alterei funcao pra retornar o merge_p, 
# ta retornando no formato de vetor mas talvez tenha q mudar para dataframe ou serie
    
    
    
    if( isinstance(file_in , str)):
        deg_list = pd.read_csv(file_in,  sep = "\t")#header true no original, so tirei o header pra o python inferir
        if(analysis_name is None):
            analysis_name =   os.path.splitext(os.path.basename(file_in) )[0] # pega o basename e tira a extensao
     
    elif(isinstance(file_in , pd.DataFrame)):
        deg_list = file_in
    

    if(analysis_name is None):
        analysis_name = "PEBBA_analysis"
    
    ## Remove rows that do not have a valid gene symbol
    deg_list = deg_list.dropna()
    ## Get background genes as a character vector
    ## Empty values (non-annotated genes) will be removed
    all_genes = deg_list["Gene.symbol"] # passar pra lista?
    
    # Get cutoff values -------------------------------------------------------
    if(verbose):
        print("Getting cutoff") # no original era message(), posso usar um print?
        
    table_cut = _get_cutoff(deg_list, logFC_col, pvalue_col, min_genes, max_genes)
    
    directions = ["up", "down" , "any"]
    
    cut_paths =[]
    dfs = []
    paths = []
    for direction in directions:
        if (verbose):
            print(direction + "\nGetting Pathways")
        df , path = _get_pathway(merge_p, term2gene, all_genes,
                            deg_list, gene_col, logFC_col,
                            pvalue_col, direction,
                            min_genes, max_genes, p_cut)
        if (verbose):
            print("Getting Pathway Cutoff")
        
        cut_path = _cutoff_path(path, p_cut, direction)
        
        cut_paths.append(cut_path)
        dfs.append(df)
        paths.append(path)
    
    
    ## Save heatmaps
    if(verbose):
        print("Saving heatmaps")
    

In [2]:
def validates_inputs(min_genes,max_genes,p_cut):

    if(min_genes < 50 or min_genes > 2900):
        sys.exit("Variable min_genes must be between 50 and 2900 genes")
          
    if(max_genes < 100 or max_genes > 3000):
        sys.exit("Variable max_genes must be between 100 and 3000 genes")  
    
    if(p_cut < 0.00001 or p_cut > 1):
        sys.exit("Variable p_cut must be between 0.00001 and 1")
        
        
        
    
         

In [1]:
def create_results_directory(results_dir):
    results_dir = os.path.abspath(results_dir)
    if not os.path.exists(results_dir): 
        os.makedirs("Results/Tables") 
        os.makedirs("Results/Heatmaps")      
    else:    
        if( not force): 
            sys.exit("Stopping analysis: ", results_dir, " already exists! Use force=True to overwrite.")

'/home/barzi/Área de Trabalho/pebba_python/Rederbe'

### copy pastes e testes aleatorios